Wir bleiben weiterhin bei dem Beispiel mit der Bücherverwaltung, jedoch werden wir einige neue Themen besprechen. Wir werden unser Projekt mit Exceptions (Ausnahmebehandlung), status codes, Objekten und mit der Validierung der request-Objekte verfeinern. Es sind also noch genügend Themen vorhanden, welche die Qualität der API-Entwicklung enorm beeinflussen.
<br>
<br>
Wir gehen nun das Projekt etwas anders an und erstellen für unsere Bücher direkt eine Klasse mit einem Konstruktor.
So sieht erstmal die Ausgangslage aus:

In [3]:
from fastapi import FastAPI, Body
from pydantic import BaseModel

app = FastAPI()

class Book(BaseModel):
    id: int
    title: str
    author: str
    description: str
    rating: int

BOOKS = [
    Book(id=1, title="Computer Science Pro", author="Jackson", description="A very nice book!", rating=5),
    Book(id=2, title="Be fast with FastAPI", author="Gandalf", description="A great book!", rating=5),
    Book(id=3, title="Master Endpoints", author="Jackson", description="An awesome book!", rating=5),
    Book(id=4, title="HP1", author="Author 1", description="Book Description", rating=2),
    Book(id=5, title="HP2", author="Author 2", description="Book Description", rating=3),
    Book(id=6, title="HP3", author="Author 3", description="Book Description", rating=1),
]

@app.get("/books")
async def read_all_books():
    return BOOKS


@app.post("/create_book")
async def create_book(book_request: Book):
    BOOKS.append(book_request)

**Erklärung des Codes:**

Hier wird etwas neues importiert was wir bisher nicht genutzt hatten:
```python
    from fastapi import FastAPI, Body
```

Body Wird verwendet, um den Inhalt des HTTP-Request (Body) zu extrahieren und zu verarbeiten. In diesem Fall hilft es dabei, die Daten des Buches zu empfangen, die in einem POST-Request gesendet werden.
<br>
<br>
Die Book-Klasse definiert das Datenmodell für ein Buch mit bestimmten Attributen. Außerdem befindet sich innerhalb der Klasse ein Konstruktor `__init__()`, welcher verwendet wird um bei der Erstellung eines buch-Objekts, die Attribute zu initialisieren.
<br>
<br>
Was bisher ebenfalls nicht genutzt wurde ist die folgende Schreibweise: `book_request=Body()`. FastAPI übernimmt die Konvertierung des Request-Bodys in ein Book-Objekt. Der Body()-Wrapper sagt FastAPI, dass die Daten aus dem HTTP-Request-Body extrahiert werden sollen.
<br>
<br>
Jetzt können wir den POST-Request testen mit den Daten:

```json
{
    "id": 7,
    "title": "HP4",
    "author": "Author 3",
    "description": "Book Description",
    "rating": 1
}
```

Mithilfe des GET-Requests sollte jetzt ein Eintrag mehr vorhanden sein. Alles funktioniert erstmal wie bisher. Doch was wenn wir nicht valide Daten bei dem POST-Request verwenden? Nehmen wir als Beispiel den folgenden Body:

```json
{
    "id": -90,
    "title": "HP4",
    "author": "Author 3",
    "description": "Book Description",
    "rating": 1999
}
```

Das Anlegen des Buches war ebenfalls erfolgreich! Wie können wir nun dieses Problem lösen? Wie brauchen also eine Art von Datenvalidierung.
Für diesen Fall hatten wir Pydantics kennen gelernt, jetzt können wir es richtig in der Praxis anwenden.